In [ ]:
"""
Update Parameters Here
"""
COLLECTION = "8_BIT_UNIVERSE"
CONTRACT = "0xaAe71BBbaA359BE0d81D5CBc9B1E88A8b7c58a94"

In [ ]:
"""
@author: mdigi14
"""

import requests
import pandas as pd

from utils import config

ETHER_UNITS = 1e18
LIMIT = 200
MAX_OFFSET = 10000
MAX_TOKEN_ID = 9999

"""
Helper Functions
"""


def get_all_sales():
    events = []
    new_events = []
    offset = 0

    while True:
        query_string = {
            "asset_contract_address": CONTRACT,
            "event_type": "successful",
            "limit": LIMIT,
            "offset": offset,
        }

        headers = {"Accept": "application/json", "X-API-KEY": config.OPENSEA_API_KEY}

        response = requests.request(
            "GET",
            url="https://api.opensea.io/api/v1/events",
            headers=headers,
            params=query_string,
        ).json()
        new_events = response["asset_events"]
        events.extend(new_events)

        if len(new_events) < LIMIT or offset > MAX_OFFSET:
            if offset > MAX_OFFSET:
                print(
                    "Final offset was",
                    offset,
                    ". May not have full sale dataset.\nSee 'find_minting_data.ipynb' for handling offsets above 10k",
                )
            break
        offset += LIMIT

    return events

In [ ]:
"""
Generate Dataset
"""
events = get_all_sales()
events_list = []

for event in events:
    event_dict = dict()

    try:
        event_dict = {
            "id": int(event["asset"]["token_id"]),
            "price": float(event["total_price"]) / ETHER_UNITS,
            "date_sold": event["transaction"]["timestamp"],
        }

        events_list.append(event_dict)
    except:
        continue

data = pd.DataFrame(events_list)
data = data.sort_values(by=["date_sold"]).drop_duplicates(
    "id", keep="first"
)  # keep first sale of a token only
data.to_csv(f"{config.FIRST_FLIP_REVENUE_FOLDER}/{COLLECTION}_first_sale_revenue.csv")